# Trend-Scanning を使った「モメンタム・ホライズン有効性」検証ノートブック

このノートブックは、既存の `trend_scanning_reproduction.ipynb`（参照版/Numba版）を読み込み、
1) `h*(t)`（最適窓）の分布と遷移
2) モメンタム（複数ホライズン）の予測力/収益性（Long-only / Long-Short / EQW）
3) Trend-Scanning によるゲーティング（局面でホライズンを切替）
を **リークを避けた手順**で評価します。

※データは `yfinance` を優先（使えない場合はCSVに差し替え）。

In [ ]:
# ===== 0) 既存ノートブックを読み込む =====
# ここで trend_scanning_labels_reference / trend_scanning_labels_numba などがロードされます
%run ./trend_scanning_reproduction.ipynb

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional


In [ ]:
# ===== 1) 設定 =====
CFG = {
    # ユニバース（例）: セクターETFなど。あなたのテーマETFユニバースに置換してください。
    "tickers": ["SPY","QQQ","IWM","EFA","EEM","TLT","IEF","GLD","USO"],
    # データ期間
    "start": "2005-01-01",
    "end": None,  # None なら today
    # リバランス頻度
    "rebalance": "M",  # "W" or "M"
    # 予測対象（フォワード）: 1か月先（営業日換算の近似）
    "fwd_days": 21,
    # モメンタム候補ホライズン（営業日）
    "mom_horizons": [21, 63, 126, 252],
    # 12Mモメンタムの「1M skip」をする場合のskip日数（例：21）
    "skip_days_for_252": 21,
    # クロスセクション戦略の top-k
    "top_k": 3,
    # 取引コスト（比例、片道）bps
    "cost_bps": 10.0,
    # Trend-Scanning span（候補窓の範囲）
    "ts_span": (10, 252),  # 10営業日〜252営業日
    # Trend-Scanning の low-vol マスク閾値（expanding quantile）
    "ts_vol_q": 0.05,
    # Trend-Scanning 実装選択: "numba" or "ref"
    "ts_impl": "numba",
    # ゲーティング閾値（窓長で短期/中期/長期に切替）
    "gate_bins": [30, 120],  # <=30 short, <=120 mid, else long
}

CFG


In [ ]:
# ===== 2) データ取得 =====
def load_prices_yfinance(tickers: List[str], start: str, end: Optional[str]) -> pd.DataFrame:
    import yfinance as yf
    df = yf.download(tickers, start=start, end=end, auto_adjust=True, progress=False)["Close"]
    if isinstance(df, pd.Series):
        df = df.to_frame()
    df = df.dropna(how="all").sort_index()
    df.columns = [c.upper() for c in df.columns]
    return df

try:
    PX = load_prices_yfinance([t.upper() for t in CFG["tickers"]], CFG["start"], CFG["end"])
    print("Loaded via yfinance:", PX.shape, PX.index.min(), PX.index.max())
except Exception as e:
    print("yfinance failed:", repr(e))
    print("-> 代替として、CSVを読み込むようにしてください（index=Date、columns=tickers）。")
    raise


In [ ]:
# ===== 3) 便利関数: リバランス・フォワードリターン・ターンオーバー =====
def to_rebalance_dates(px: pd.DataFrame, freq: str) -> pd.DatetimeIndex:
    if freq == "M":
        return px.resample("M").last().index
    if freq == "W":
        return px.resample("W-FRI").last().index
    raise ValueError("rebalance must be 'W' or 'M'")

def forward_return(px: pd.DataFrame, fwd_days: int) -> pd.DataFrame:
    # t -> t+fwd_days のリターン（リーク回避のため、評価側で shift を管理）
    return px.pct_change(fwd_days).shift(-fwd_days)

def normalize_weights(w: pd.DataFrame) -> pd.DataFrame:
    s = w.abs().sum(axis=1).replace(0.0, np.nan)
    return w.div(s, axis=0).fillna(0.0)

def turnover(w: pd.DataFrame) -> pd.Series:
    # 片側合計回転（L1 turnover / 2 の解釈もあるが、ここではL1をそのまま出す）
    dw = w.diff().abs().sum(axis=1)
    return dw.fillna(0.0)

def apply_costs(port_ret: pd.Series, w: pd.DataFrame, cost_bps: float) -> pd.Series:
    tc = (cost_bps / 1e4) * turnover(w)
    return port_ret - tc

def portfolio_return(w: pd.DataFrame, fwd_ret: pd.DataFrame) -> pd.Series:
    # 同日時点で掛ける（wはtで決めてt->t+fwdのリターンに乗る想定）
    common = w.index.intersection(fwd_ret.index)
    ww = w.loc[common]
    rr = fwd_ret.loc[common, ww.columns]
    return (ww * rr).sum(axis=1)


## 4) Trend-Scanningで `h*(t)` を作る（**lookforward=False** で状態変数化）
- ここは“ホライズン選択器”として使うため、**未来を見ない**（lookforward=False）。
- `window`（最適窓）を取り出して、後段のゲーティングに使います。

In [ ]:
def trend_scanning_windows(close: pd.Series) -> pd.DataFrame:
    impl = CFG["ts_impl"]
    fn = trend_scanning_labels_numba if impl == "numba" else trend_scanning_labels_reference
    df = fn(
        close,
        span=CFG["ts_span"],
        lookforward=False,        # 重要：過去のみ
        use_log=True,
        volatility_threshold=CFG["ts_vol_q"],
    )
    # 返り値は t1 等もあるが、ここでは window と t_value を主に使う
    return df

# 例としてベンチマーク（SPY）で window を作る
BENCH = "SPY" if "SPY" in PX.columns else PX.columns[0]
ts_bench = trend_scanning_windows(PX[BENCH].dropna())

ts_bench.head()


In [ ]:
# 分布と時系列
plt.figure()
ts_bench["window"].dropna().hist(bins=40)
plt.title(f"Trend-Scanning optimal window distribution ({BENCH})")
plt.xlabel("window (trading days)")
plt.ylabel("count")
plt.show()

plt.figure()
ts_bench["window"].dropna().plot()
plt.title(f"Trend-Scanning optimal window over time ({BENCH})")
plt.xlabel("date")
plt.ylabel("window (days)")
plt.show()


## 5) モメンタム信号（複数ホライズン）の作成

ここでは、ETFユニバースに対して
- **Time-series momentum（各ETFの符号）**
- **Cross-sectional momentum（ランキング）**
の両方を作って、ホライズン別に比較します。

**重要**: 信号は必ず `t` までの情報で作り、運用に使うのは `t+1`（次期）から、を徹底します（`shift(1)`）。

In [ ]:
def mom_signal_timeseries(px: pd.DataFrame, h: int) -> pd.DataFrame:
    # sign( past return over h )
    ret = px.pct_change(h)
    sig = np.sign(ret)
    return sig

def mom_signal_crosssection(px: pd.DataFrame, h: int) -> pd.DataFrame:
    # cross-sectional rank of past return (higher = better)
    ret = px.pct_change(h)
    # rank in [0,1]
    rk = ret.rank(axis=1, method="average", pct=True)
    # centered rank in [-0.5, +0.5]
    return rk - 0.5

def mom_signal_12m_skip(px: pd.DataFrame, h: int = 252, skip: int = 21) -> pd.DataFrame:
    # (t-skip) / (t-h) - 1
    # 252d momentum skipping most recent 21d
    a = px.shift(skip)
    ret = a.pct_change(h - skip)
    return np.sign(ret)

# 作成
MOM_TS: Dict[int, pd.DataFrame] = {}
MOM_CS: Dict[int, pd.DataFrame] = {}

for h in CFG["mom_horizons"]:
    if h == 252 and CFG["skip_days_for_252"] is not None:
        MOM_TS[h] = mom_signal_12m_skip(PX, h=252, skip=CFG["skip_days_for_252"])
        MOM_CS[h] = mom_signal_crosssection(PX.shift(CFG["skip_days_for_252"]), h=252-CFG["skip_days_for_252"])
    else:
        MOM_TS[h] = mom_signal_timeseries(PX, h)
        MOM_CS[h] = mom_signal_crosssection(PX, h)

list(MOM_TS.keys()), list(MOM_CS.keys())


## 6) ポートフォリオ構築（Long-only / Long-Short / EQW）

Cross-sectional momentum を使って
- Long-only: top-k 等ウェイト
- Long-short: top-k をロング、bottom-k をショート（ドルニュートラル）
を作り、EQWも併記します。

In [ ]:
def weights_topk_longonly(score: pd.DataFrame, top_k: int) -> pd.DataFrame:
    # scoreが高い上位top_kを等ウェイト（0/1）
    ranks = score.rank(axis=1, ascending=False, method="first")
    w = (ranks <= top_k).astype(float)
    w = w.div(w.sum(axis=1).replace(0.0, np.nan), axis=0).fillna(0.0)
    return w

def weights_topk_longshort(score: pd.DataFrame, k: int) -> pd.DataFrame:
    ranks_hi = score.rank(axis=1, ascending=False, method="first")
    ranks_lo = score.rank(axis=1, ascending=True, method="first")
    long = (ranks_hi <= k).astype(float)
    short = (ranks_lo <= k).astype(float) * (-1.0)
    w = long + short
    # gross=1 に正規化
    w = normalize_weights(w)
    return w

def weights_eqw(px: pd.DataFrame) -> pd.DataFrame:
    w = pd.DataFrame(1.0, index=px.index, columns=px.columns)
    w = w.div(w.sum(axis=1), axis=0)
    return w


## 7) 評価（ホライズン別）

ここでは **評価ターゲットを次の1か月（約21営業日）** とし、
- IC（スコアと次期リターンの相関）
- hit rate（ロングが勝った比率）
- コスト込み累積リターン
を出します。

※コストは turnover×bps（比例）

In [ ]:
def monthly_rebalance_weights(w_daily: pd.DataFrame, px: pd.DataFrame, freq: str) -> pd.DataFrame:
    # リバランス日で決めて、次リバランス日までホールド
    rb = to_rebalance_dates(px, freq)
    rb = rb.intersection(w_daily.index)
    w_rb = w_daily.loc[rb]
    # 実運用：月末決定→翌営業日から適用（リーク防止の要）
    w_rb = w_rb.shift(1)
    w_hold = w_rb.reindex(px.index, method="ffill").fillna(0.0)
    return w_hold

def compute_ic(score: pd.DataFrame, fwd_ret: pd.DataFrame, freq: str) -> pd.Series:
    rb = to_rebalance_dates(score, freq).intersection(score.index).intersection(fwd_ret.index)
    # 月末のスコアで、次月(=fwd_ret)を評価（スコアは t、リターンは t->t+fwd）
    s = score.loc[rb]
    r = fwd_ret.loc[rb, s.columns]
    ic = s.corrwith(r, axis=1)
    return ic

def summarize_perf(port_ret: pd.Series) -> pd.Series:
    port_ret = port_ret.dropna()
    if len(port_ret) == 0:
        return pd.Series({"mean": np.nan, "vol": np.nan, "sharpe": np.nan})
    mu = port_ret.mean()
    sd = port_ret.std(ddof=0)
    sharpe = mu / sd if sd > 0 else np.nan
    return pd.Series({"mean": mu, "vol": sd, "sharpe": sharpe})


In [ ]:
FWD = forward_return(PX, CFG["fwd_days"])

results = []

# EQW
w_eqw = monthly_rebalance_weights(weights_eqw(PX), PX, CFG["rebalance"])
ret_eqw = portfolio_return(w_eqw, FWD)
ret_eqw_c = apply_costs(ret_eqw, w_eqw, CFG["cost_bps"])
results.append(("EQW", "NA", "EQW", summarize_perf(ret_eqw_c)))

# horizon別
for h in CFG["mom_horizons"]:
    score = MOM_CS[h]
    ic = compute_ic(score, FWD, CFG["rebalance"])

    # long-only
    w_lo = weights_topk_longonly(score, CFG["top_k"])
    w_lo = monthly_rebalance_weights(w_lo, PX, CFG["rebalance"])
    r_lo = portfolio_return(w_lo, FWD)
    r_lo_c = apply_costs(r_lo, w_lo, CFG["cost_bps"])

    # long-short
    w_ls = weights_topk_longshort(score, CFG["top_k"])
    w_ls = monthly_rebalance_weights(w_ls, PX, CFG["rebalance"])
    r_ls = portfolio_return(w_ls, FWD)
    r_ls_c = apply_costs(r_ls, w_ls, CFG["cost_bps"])

    results.append((f"CSMOM_{h}", h, "LongOnly", summarize_perf(r_lo_c).assign(IC=ic.mean())))
    results.append((f"CSMOM_{h}", h, "LongShort", summarize_perf(r_ls_c).assign(IC=ic.mean())))

summary = pd.DataFrame([{"name":n, "h":h, "style":sty, **met.to_dict()} for (n,h,sty,met) in results])
summary.sort_values(["style","sharpe"], ascending=[True, False]).reset_index(drop=True)


In [ ]:
# 累積リターンをプロット（上位いくつか）
def cumplot(series_dict: Dict[str, pd.Series], title: str):
    plt.figure()
    for k, s in series_dict.items():
        cs = (1.0 + s.fillna(0.0)).cumprod()
        cs.plot(label=k)
    plt.title(title)
    plt.xlabel("date")
    plt.ylabel("cumulative")
    plt.legend()
    plt.show()

# 例：long-onlyの比較
rets = {"EQW": ret_eqw_c}
for h in CFG["mom_horizons"]:
    score = MOM_CS[h]
    w_lo = monthly_rebalance_weights(weights_topk_longonly(score, CFG["top_k"]), PX, CFG["rebalance"])
    r_lo_c = apply_costs(portfolio_return(w_lo, FWD), w_lo, CFG["cost_bps"])
    rets[f"CSMOM{h}_LO"] = r_lo_c

cumplot(rets, f"Long-only (top-{CFG['top_k']}) cumulative returns (cost {CFG['cost_bps']} bps)")


## 8) Trend-Scanning ゲーティング：局面でホライズンを切り替える

ここでは `BENCH` の `window=h*(t)` を使って、
- short regime: window<=gate_bins[0] → 21d
- mid regime: window<=gate_bins[1] → 63d
- long regime: else → 252d
のようにホライズンを切替。

重要：`window` は lookforward=False で作っているので、ゲーティング自体はリークしません。

In [ ]:
def gate_horizon_from_window(window_series: pd.Series, gate_bins: List[int], horizons: List[int]) -> pd.Series:
    # horizonsは [short, mid, long] の3つを想定
    assert len(horizons) == 3
    b0, b1 = gate_bins
    w = window_series.copy()
    out = pd.Series(index=w.index, dtype=float)
    out[w <= b0] = horizons[0]
    out[(w > b0) & (w <= b1)] = horizons[1]
    out[w > b1] = horizons[2]
    return out

# BENCHのwindowをPXのindexに合わせる
win = ts_bench["window"].reindex(PX.index).ffill()
gate_h = gate_horizon_from_window(win, CFG["gate_bins"], [21, 63, 252])

gate_h.value_counts().sort_index()


In [ ]:
# ゲートされたスコア（時点ごとに異なるホライズンのCSMOMスコアを使う）
def gated_score(score_by_h: Dict[int, pd.DataFrame], gate_h: pd.Series) -> pd.DataFrame:
    idx = score_by_h[list(score_by_h.keys())[0]].index.intersection(gate_h.index)
    cols = score_by_h[list(score_by_h.keys())[0]].columns
    out = pd.DataFrame(index=idx, columns=cols, dtype=float)
    for h, sc in score_by_h.items():
        mask = (gate_h.loc[idx] == h).to_numpy()
        if mask.any():
            out.loc[idx[mask], :] = sc.loc[idx[mask], :].values
    return out

g_score = gated_score(MOM_CS, gate_h)

# ゲート戦略：long-only / long-short
w_g_lo = monthly_rebalance_weights(weights_topk_longonly(g_score, CFG["top_k"]), PX, CFG["rebalance"])
r_g_lo_c = apply_costs(portfolio_return(w_g_lo, FWD), w_g_lo, CFG["cost_bps"])

w_g_ls = monthly_rebalance_weights(weights_topk_longshort(g_score, CFG["top_k"]), PX, CFG["rebalance"])
r_g_ls_c = apply_costs(portfolio_return(w_g_ls, FWD), w_g_ls, CFG["cost_bps"])

# 比較プロット
rets_gate = {"EQW": ret_eqw_c, "GATED_LO": r_g_lo_c, "GATED_LS": r_g_ls_c}
cumplot(rets_gate, "Gated strategy vs EQW (cost-adjusted)")

pd.DataFrame({
    "EQW": summarize_perf(ret_eqw_c),
    "GATED_LO": summarize_perf(r_g_lo_c),
    "GATED_LS": summarize_perf(r_g_ls_c),
})


## 9) 批判的チェック（必須）

1. **multiple testing**：候補窓が多いほど `|t|` 最大化は過剰適応しやすい → 候補窓を粗くして再検証
2. **目的関数のズレ**：`|t|` 最大化は予測力最大化ではない → IC/P&Lで別途検証（上で実施）
3. **コスト耐性**：ゲーティングが短期を多用するとturnover増 → turnover分布を必ず比較
4. **リーク監査**：windowは lookforward=False / ウェイトは shift(1) / リターンは fwd を shift(-fwd) のみ、を確認


In [ ]:
# turnover比較
turn = pd.DataFrame({
    "EQW": turnover(w_eqw),
    "GATED_LO": turnover(w_g_lo),
    "GATED_LS": turnover(w_g_ls),
})
turn.describe().T


In [ ]:
plt.figure()
turn["GATED_LO"].hist(bins=40)
plt.title("Turnover distribution (GATED_LO)")
plt.xlabel("L1 turnover")
plt.ylabel("count")
plt.show()
